In [1]:
############################################################
#Run this cell to gain access to engine_helpers.py functions
############################################################

#clone repository and input helper functions to the notebook
!git clone https://github.com/jmarchese1/poker-ai-headsup.git
%cd /content/poker-ai-headsup

#best practice to only import the necessacary functions
from engine.engine_helpers import preflop_hand_strength, create_deck, deal_cards, clear_hand, clear_board, get_seven_card_hand, evaluate_hand, best_hand, has_straight_draw, has_flush_draw, flop, turn, river

Cloning into 'poker-ai-headsup'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 49 (delta 14), reused 36 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 53.20 KiB | 13.30 MiB/s, done.
Resolving deltas: 100% (14/14), done.
/content/poker-ai-headsup


In [2]:
#testing that function works properly
preflop_hand_strength(("7", "spades"), ("Ace", "clubs"))

0.591

**Pokerbot decision making, logic, and randomness**

In [3]:
#creating a poker bot class to make decisions depending on specific circumstances in the core_engine
import random

class PokerBot:
  """
  This class creates a bot to play the small blind position in poker
  """
  def __init__(self, name, stack_size, small_blind, big_blind):
    self.name = name #name of the bot
    self.chips = stack_size #amount of chips the bot has to play with
    self.hand = [] #the gameplay engine automatically deals players the hand.
    self.small_blind = small_blind #expected 1
    self.big_blind = big_blind #expected 2


  def get_preflop_decision(self):
    """
    Bot A decision making for getting "pre_flop_bota_hand_strength" and "pre_flop_bota_decision"
    """
    strength = preflop_hand_strength(self.hand[0], self.hand[1])
    if strength > 0.8:
      random_number = random.randint(1, 100)
      if random_number >= 7:
        preflop_decision = "raise"
      else:
        preflop_decision = "call"
    elif strength > 0.6 and strength <= 0.8:
      random_number = random.randint(1, 100)
      if random_number >= 25:
        preflop_decision = "raise"
      else:
        preflop_decision = "call"
    elif strength > 0.4 and strength <= 0.6:
      random_number = random.randint(1, 100)
      if random_number >= 70:
        preflop_decision = "raise"
      else:
        preflop_decision = "call"
    elif strength > 0 and strength <= 0.4:
      random_number = random.randint(1, 100)
      if random_number >= 90:
        preflop_decision = "raise"
      elif random_number >= 15:
        preflop_decision = "fold"
      else:
        preflop_decision = "call"

    return preflop_decision, strength

  #game set up for players to have pretty much ultimited chips for
  def get_preflop_raise_amount(self):
    "Bot A preflop raise amount"
    random_number = random.randint(1, 100)
    if random_number >= 95:
      preflop_raise_amount = 4 * self.big_blind
    elif random_number >= 50:
      preflop_raise_amount = 3 * self.big_blind
    else:
      preflop_raise_amount = 2 * self.big_blind

    return preflop_raise_amount

  #if the other player re-raises, Bot A needs to make another decision
  def get_preflop_decision_2(self):
    """
    Bot A decision making for responding to other players re-raise
    """
    #need to refer to preflop hand strength again
    strength = preflop_hand_strength(self.hand[0], self.hand[1])

    #now the bot will act skeptical assuming the other player has a very strong hand
    #lets assume all big blind re-raises are 3x bot A's bet
    if strength > 0.4:
      preflop_decision_2 = "call"
    else:
      random_number = random.randint(1, 100)
      if random_number >= 50:
        preflop_decision_2 = "call"
      else:
        preflop_decision_2 = "fold"

    return preflop_decision_2

  #the next decision to be made is the first post flop decision
  def get_post_flop_decision(self, board):
    """
    Bot A decision making for its inital post flop action choice

    either check or bet.
    """
    #splitting suits and numbers to aid in the decision making process and bot logic

    #this sequence determines what state the players hand is in -- it would be helpful to update the function to recognize board patterns and make more complex decisions
    full_hand = get_seven_card_hand(self, board) #returns a list of the players cards in addition to whats displayed on the board
    hand_ranking = evaluate_hand(full_hand)

    #see if there is more to the bots hand than whats visible
    flush_draw = has_flush_draw(full_hand) #function will return True if the bot needs one card to make a flush
    straight_draw = has_straight_draw(full_hand) #functiono will return True if the bot needs one card to make a straight

    #how the bot will act when it flops nothing
    if hand_ranking[0] == 1 and flush_draw == False and straight_draw == False:
      random_number = random.randint(1,100)
      if random_number >= 15:
        post_flop_decision = "check"
      else:
        post_flop_decision = "bet"
    #how the bot will act when it flops air with a flush or straight draw
    elif hand_ranking[0] == 1 and flush_draw == True or straight_draw == True:
      if random.randint(1, 100) > 40:
        post_flop_decision = "check"
      else:
        post_flop_decision = "bet"
    #how the bot will act when it flops a pair
    elif hand_ranking[0] == 2:
      pair_strength = hand_ranking[5]
      if pair_strength == 1:
        if random.randint(1, 100) > 30:
          post_flop_decision = "bet"
        else:
          post_flop_decision = "check"
      elif pair_strength == 2:
        if random.randint(1, 100) > 40:
          post_flop_decision = "bet"
        else:
          post_flop_decision = "check"
      else:
        if random.randint(1, 100) > 55:
          post_flop_decision = "bet"
        else:
          post_flop_decision = "check"
    #if the hand ranking is greater then one pair -- can assume the bot flopped an amazing hand and all should play similarly
    else:
      if random.randint(1, 100) > 50:
        post_flop_decision = "bet"
      else:
        post_flop_decision = "check"

    return post_flop_decision, hand_ranking[0]

  def get_post_flop_bet_amount(self, pot, hand_ranking): #this function needs to know the potsize to determine sizings
    """
    Bot A post flop bet amount
    """
    if hand_ranking == 1:
      if random.randint(1, 100) > 50:
        post_flop_bet_amount = int(round(1/3 * pot))
      else:
        post_flop_bet_amount = int(round(1/2 * pot))
    elif hand_ranking == 2:
      if random.randint(1, 100) > 70:
        post_flop_bet_amount = int(round(1/3 * pot))
      else:
        post_flop_bet_amount = int(round(1/2 * pot))
    else:
      if random.randint(1, 100) > 50:
        post_flop_bet_amount = int(round(1/2 * pot))
      else:
        post_flop_bet_amount = int(round(2/3 * pot))

    return post_flop_bet_amount

  def get_post_flop_decision_2(self, board):
    """
    This function is for if the other player re-raises a bet, or if the other player bets after bot-a checks
    """
    #this sequence determines what state the players hand is in -- it would be helpful to update the function to recognize board patterns and make more complex decisions
    full_hand = get_seven_card_hand(self, board) #returns a list of the players cards in addition to whats displayed on the board
    hand_ranking = evaluate_hand(full_hand)

    #see if there is more to the bots hand than whats visible
    flush_draw = has_flush_draw(full_hand) #function will return True if the bot needs one card to make a flush
    straight_draw = has_straight_draw(full_hand) #functiono will return True if the bot needs one card to make a straight

    if hand_ranking[0] == 1 and flush_draw == False and straight_draw == False:
        post_flop_decision = "fold"
    #how the bot will act when it flops air with a flush or straight draw
    elif hand_ranking[0] == 1 and flush_draw == True or straight_draw == True:
      if random.randint(1, 100) > 50:
        post_flop_decision = "call"
      else:
        post_flop_decision = "fold"
    #how the bot will act when it flops a pair
    elif hand_ranking[0] == 2:
      pair_strength = hand_ranking[5]
      if pair_strength == 1:
        if random.randint(1, 100) > 30:
          post_flop_decision = "call"
        else:
          post_flop_decision = "re-raise"
      elif pair_strength == 2:
        if random.randint(1, 100) > 10:
          post_flop_decision = "call"
        else:
          post_flop_decision = "re-raise"
      else:
        if random.randint(1, 100) > 20:
          post_flop_decision = "call"
        else:
          post_flop_decision = "fold"
    #if the hand ranking is greater then one pair -- can assume the bot flopped an amazing hand and all should play similarly
    else:
      if random.randint(1, 100) > 25:
        post_flop_decision = "call"
      else:
        post_flop_decision = "re-raise"

    return post_flop_decision, hand_ranking[0]

  def get_re_raise_amount(self, opponent_bet_amount, hand_ranking):
    """
    This function is for how much Bot-a will re-raise the opponent's bet on any street.
    """
    if hand_ranking == 2:
      if random.randint(1, 100) > 50:
        re_raise_amount = 2 * opponent_bet_amount
      else:
        re_raise_amount = 2.5 * opponent_bet_amount

    else:
      random_number = random.randint(1, 100)
      if random_number > 50:
        re_raise_amount = 3 * opponent_bet_amount
      elif random_number > 10:
        re_raise_amount = 2.5 * opponent_bet_amount
      else:
        re_raise_amount = 2 * opponent_bet_amount

    return re_raise_amount

    #the hand history starts to matter more at the turn so its important to consider if the opponent checked or bet on the flop for more accurate decision making
    def get_turn_decision(self, board, opponent_post_flop_decision, opponent_post_flop_decision_2 = None):
      """
      Bot A decision making for its turn action choice, based on the strength of its hand and opponents decision history
      """
        #this sequence determines what state the players hand is in -- it would be helpful to update the function to recognize board patterns and make more complex decisions
      full_hand = get_seven_card_hand(self, board) #returns a list of the players cards in addition to whats displayed on the board
      hand_ranking = evaluate_hand(full_hand)

      #see if there is more to the bots hand than whats visible
      flush_draw = has_flush_draw(full_hand) #function will return True if the bot needs one card to make a flush
      straight_draw = has_straight_draw(full_hand) #functiono will return True if the bot needs one card to make a straight

      #this means it went check check -- could mean the opponent is weak
      if opponent_post_flop_decision == "check":
        #if the bot made it this long with air -- it might be a good idea to start semi bluffing
        if hand_ranking[0] == 1 and flush_draw == False and straight_draw == False:
            random_number = random.randint(1, 100)
            if random_number >= 55: #more than likely the bot will bet on the turn if it went check-check on the flop
              post_turn_decision = "check"
            else:
              post_turn_decision = "bet"

        #how the bot will act when it flops air with a flush or straight draw, will keep the odds the same as air
        elif hand_ranking[0] == 1 and flush_draw == True or straight_draw == True:
          if random.randint(1, 100) > 55:
            post_turn_decision = "check"
          else:
            post_turn_decision = "bet"
        #if it went check-check its likely the bot flopped its pair on the turn -- meaning its a strong time to bet
        elif hand_ranking[0] == 2:
          pair_strength = hand_ranking[5]
          if pair_strength == 1:
            if random.randint(1, 100) > 15:
              post_turn_decision = "bet"
            else:
              post_turn_decision = "check"
          elif pair_strength == 2:
            if random.randint(1, 100) > 35:
              post_turn_decision = "bet" #need to be careful if the opponent is slow playing top pair on the flop
            else:
              post_turn_decision = "check"
          else:
            if random.randint(1, 100) > 50: #can potentially be the best hand -- but also can be getting trapped, hopefully opponent folds to this bet
              post_turn_decision = "bet"
            else:
              post_turn_decision = "check"
        #if the hand ranking is greater then one pair -- can assume the bot flopped an amazing hand and all should play similarly
        else:
          if random.randint(1, 100) > 25:
            post_turn_decision = "bet" #good time to start building a pot with strong hands
          else:
            post_turn_decision = "check"

        return post_turn_decision, hand_ranking[0]

      #this means that bot_a checked to the opponent and they decided to bet -- potential sign of strength or they are just trying to take down the pot with nothing
      elif opponent_post_flop_decision == "bet":
        #if the bot made it this long with air -- it might be a good idea to start semi bluffing
        if hand_ranking[0] == 1 and flush_draw == False and straight_draw == False:
            random_number = random.randint(1, 100)
            if random_number >= 15: #really should be checking here -- no reason to bet
              post_turn_decision = "check"
            else:
              post_turn_decision = "bet"

        #how the bot will act when it flops air with a flush or straight draw
        elif hand_ranking[0] == 1 and flush_draw == True or straight_draw == True:
          if random.randint(1, 100) > 75: #could be a good semi bluff spot
            post_turn_decision = "bet"
          else:
            post_turn_decision = "check"
        #how the bot will act when has a pair on the turn
        elif hand_ranking[0] == 2:
          pair_strength = hand_ranking[5]
          if pair_strength == 1:
            if random.randint(1, 100) > 20:
              post_turn_decision = "bet"
            else:
              post_turn_decision = "check"
          elif pair_strength == 2:
            if random.randint(1, 100) > 40:
              post_turn_decision = "bet"
            else:
              post_turn_decision = "check"
          else:
            if random.randint(1, 100) > 65:
              post_turn_decision = "bet"
            else:
              post_turn_decision = "check"
        #if the hand ranking is greater then one pair -- can assume the bot flopped an amazing hand and all should play similarly
        else:
          if random.randint(1, 100) > 50:
            post_flop_decision = "bet"
          else:
            post_turn_decision = "check"

        return post_turn_decision, hand_ranking[0]

        #this means the opponents postflop decision was to re-raise (this means we should be skeptical of them having a strong hand)
      else:
          #if the bot made it this long with air -- it might be a good idea to start semi bluffing
        if hand_ranking[0] == 1 and flush_draw == False and straight_draw == False:
            post_turn_decision = "check"

        #how the bot will act when it flops air with a flush or straight draw
        elif hand_ranking[0] == 1 and flush_draw == True or straight_draw == True:
          post_turn_decision = "check"
        #how the bot will act when it flops a pair

        elif hand_ranking[0] == 2:
          pair_strength = hand_ranking[5]
          if pair_strength == 1:
            if random.randint(1, 100) > 50: #dangerous against a re-raise
              post_turn_decision = "bet"
            else:
              post_turn_decision = "check"
          elif pair_strength == 2:
            if random.randint(1, 100) > 75:
              post_turn_decision = "bet"
            else:
              post_turn_decision = "check"
          else:
            if random.randint(1, 100) > 90: #very weak pair -- probably behind here
              post_turn_decision = "bet"
            else:
              post_turn_decision = "check"

        #hands greater than a pair should be less cautious but also skeptical of the opponent - a lead out bet here is probaly best to show strength through the re-raise
        else:
          if random.randint(1, 100) > 25:
            post_turn_decision = "bet"
          else:
            post_flop_decision = "check"

        return post_turn_decision, hand_ranking[0]

  def get_post_turn_bet_amount(self, pot, hand_ranking): #this function needs to know the potsize to determine sizings
    """
    Bot A post turn bet amount
    """

    #at this point the terrible hand should start acting as a bluff
    if hand_ranking == 1:
      if random.randint(1, 100) > 50:
        post_flop_bet_amount = int(round(3/4 * pot)) #large bets to get folds
      else:
        post_flop_bet_amount = int(round(1 * pot))
    elif hand_ranking == 2:
      if random.randint(1, 100) > 70:
        post_flop_bet_amount = int(round(1/3 * pot))
      else:
        post_flop_bet_amount = int(round(1/2 * pot))
    else:
      if random.randint(1, 100) > 50:
        post_flop_bet_amount = int(round(1/2 * pot))
      else:
        post_flop_bet_amount = int(round(1 * pot))

    return post_flop_bet_amount

  def get_post_turn_decision_2(self, board):
    """
    This function is for if the other player re-raises a bet, or if the other player bets after bot-a checks on the turn
    """
    #this sequence determines what state the players hand is in -- it would be helpful to update the function to recognize board patterns and make more complex decisions
    full_hand = get_seven_card_hand(self, board) #returns a list of the players cards in addition to whats displayed on the board
    hand_ranking = evaluate_hand(full_hand)

    #see if there is more to the bots hand than whats visible
    flush_draw = has_flush_draw(full_hand) #function will return True if the bot needs one card to make a flush
    straight_draw = has_straight_draw(full_hand) #function will return True if the bot needs one card to make a straight

    if hand_ranking[0] == 1 and flush_draw == False and straight_draw == False: #opponent is showing a lot of strength -- fold air
        post_turn_decision = "fold"
    #how the bot will act when it turns air with a flush or straight draw
    elif hand_ranking[0] == 1 and flush_draw == True or straight_draw == True: #most likely priced in here to call a flush draw
      if random.randint(1, 100) > 20:
        post_turn_decision = "call"
      else:
        post_turn_decision = "fold"
    #how the bot will act when it turns a pair
    elif hand_ranking[0] == 2: #facing a re-raise or bet here with top pair should skeptically call
      pair_strength = hand_ranking[5]
      if pair_strength == 1:
        if random.randint(1, 100) > 20:
          post_turn_decision = "call"
        else:
          post_turn_decision = "fold"
      elif pair_strength == 2:
        if random.randint(1, 100) > 60: #usually would be behind in this scenario
          post_turn_decision = "call"
        else:
          post_turn_decision = "fold"
      else:
        if random.randint(1, 100) > 20: #low pair almost certianly losing
          post_turn_decision = "fold"
        else:
          post_turn_decision = "call"
    #if the hand ranking is greater then one pair -- logical to defend the hand in most cases with occasional re-raises
    else:
      if random.randint(1, 100) > 25:
        post_turn_decision = "call"
      else:
        post_turn_decision = "re-raise"

    return post_turn_decision, hand_ranking[0]


  #takes opponents full history of decision making into account
  def get_river_decision(self, board, opponent_post_flop_decision, opponent_post_turn_decision):
        """
        Bot A decision making for its turn action choice, based on the strength of its hand and opponents decision history
        """
          #this sequence determines what state the players hand is in -- it would be helpful to update the function to recognize board patterns and make more complex decisions
        full_hand = get_seven_card_hand(self, board) #returns a list of the players cards in addition to whats displayed on the board
        hand_ranking = evaluate_hand(full_hand)

        #see if there is more to the bots hand than whats visible
        flush_draw = has_flush_draw(full_hand) #function will return True if the bot needs one card to make a flush
        straight_draw = has_straight_draw(full_hand) #functiono will return True if the bot needs one card to make a straight

        #the opponent checked on the flop and the turn
        if opponent_post_flop_decision == "check" and opponent_post_turn_decision == "check": #the opponent is likely very weak -- worth taking a stab at winning the pot with weaker hands that would lose a showdown
          #if the bot made it this long with air -- it might be a good idea to start semi bluffing
          if hand_ranking[0] == 1 and flush_draw == False and straight_draw == False:
              random_number = random.randint(1, 100)
              if random_number >= 25: #more than likely the bot will bet on the river
                post_river_decision = "bet"
              else:
                post_river_decision = "check"

          #if opponent went check-check on turn and river its likely the bot flopped its pair on the turn or river-- meaning its a strong time to bet
          elif hand_ranking[0] == 2:
            pair_strength = hand_ranking[5]
            if pair_strength == 1:
              if random.randint(1, 100) > 10:
                post_river_decision = "bet"
              else:
                post_river_decision = "check"
            elif pair_strength == 2:
              if random.randint(1, 100) > 25:
                post_river_decision = "bet" #need to be careful if the opponent is slow playing top pair on the flop
              else:
                post_river_decision = "check"
            else:
              if random.randint(1, 100) > 65: #can potentially be the best hand -- but also can be getting trapped, hopefully opponent folds to this bet
                post_river_decision = "bet"
              else:
                post_river_decision = "check"
          #if the hand ranking is greater then one pair -- can assume the bot flopped an amazing hand and all should play similarly
          else:
            post_river_decision = "bet" #good time to start building a pot with strong hands


          return post_river_decision, hand_ranking[0]

        #if the opponent bet at each street they are most likely strong -- probably a good idea to only bet strong hands
        elif opponent_post_flop_decision == "bet" and opponent_post_turn_decision == "bet":
           #if the bot made it this long with air -- it might be a good idea to start semi bluffing
          if hand_ranking[0] == 1 and flush_draw == False and straight_draw == False:
              random_number = random.randint(1, 100)
              if random_number >= 10: #more than likely the bot will bet on the river
                post_river_decision = "check"
              else:
                post_river_decision = "bet"

          #if opponent went check-check on turn and river its likely the bot flopped its pair on the turn or river-- meaning its a strong time to bet
          elif hand_ranking[0] == 2:
            pair_strength = hand_ranking[5]
            if pair_strength == 1:
              if random.randint(1, 100) > 45:
                post_river_decision = "bet"
              else:
                post_river_decision = "check"
            elif pair_strength == 2:
              if random.randint(1, 100) > 25:
                post_river_decision = "bet" #need to be careful if the opponent is slow playing top pair on the flop
              else:
                post_river_decision = "check"
            else:
              if random.randint(1, 100) > 25: #can potentially be the best hand -- but also can be getting trapped, hopefully opponent folds to this bet
                post_river_decision = "bet"
              else:
                post_river_decision = "check"
          #if the hand ranking is greater then one pair -- can assume the bot flopped an amazing hand and all should play similarly
          else:
            post_river_decision = "bet" #good time to start building a pot with strong hands


          return post_river_decision, hand_ranking[0]

        elif opponent_post_flop_decision == "bet" and opponent_post_turn_decision == "check":
          #if the bot made it this long with air -- it might be a good idea to start semi bluffing
          if hand_ranking[0] == 1 and flush_draw == False and straight_draw == False:
              random_number = random.randint(1, 100)
              if random_number >= 70: #more than likely the bot will bet on the river
                post_river_decision = "bet"
              else:
                post_river_decision = "check"

          #bot has a pair on the river after turn and the opponent checks
          elif hand_ranking[0] == 2:
            pair_strength = hand_ranking[5]
            if pair_strength == 1:
              if random.randint(1, 100) > 25:
                post_river_decision = "bet"
              else:
                post_river_decision = "check"
            elif pair_strength == 2:
              if random.randint(1, 100) > 37:
                post_river_decision = "bet" #need to be careful if the opponent is slow playing top pair on the flop
              else:
                post_river_decision = "check"
            else:
              if random.randint(1, 100) > 55: #can potentially be the best hand -- but also can be getting trapped, hopefully opponent folds to this bet
                post_river_decision = "bet"
              else:
                post_river_decision = "check"
          #if the hand ranking is greater then one pair -- can assume the bot flopped an amazing hand and all should play similarly
          else:
            post_river_decision = "bet" #good time to start building a pot with strong hands


          return post_river_decision, hand_ranking[0]

        elif opponent_post_flop_decision == "bet" and opponent_post_turn_decision == "check":
          #if the bot made it this long with air -- it might be a good idea to start semi bluffing
          if hand_ranking[0] == 1 and flush_draw == False and straight_draw == False:
              random_number = random.randint(1, 100)
              if random_number >= 70: #more than likely the bot will bet on the river
                post_river_decision = "bet"
              else:
                post_river_decision = "check"

          #bot has a pair on the river after turn and the opponent checks
          elif hand_ranking[0] == 2:
            pair_strength = hand_ranking[5]
            if pair_strength == 1:
              if random.randint(1, 100) > 25:
                post_river_decision = "bet"
              else:
                post_river_decision = "check"
            elif pair_strength == 2:
              if random.randint(1, 100) > 37:
                post_river_decision = "bet" #need to be careful if the opponent is slow playing top pair on the flop
              else:
                post_river_decision = "check"
            else:
              if random.randint(1, 100) > 55: #can potentially be the best hand -- but also can be getting trapped, hopefully opponent folds to this bet
                post_river_decision = "bet"
              else:
                post_river_decision = "check"
          #if the hand ranking is greater then one pair -- can assume the bot flopped an amazing hand and all should play similarly
          else:
            post_river_decision = "bet" #good time to start building a pot with strong hands


          return post_river_decision, hand_ranking[0]


        elif opponent_post_flop_decision == "check" and opponent_post_turn_decision == "bet":
          #if the bot made it this long with air -- it might be a good idea to start semi bluffing
          if hand_ranking[0] == 1 and flush_draw == False and straight_draw == False:
              random_number = random.randint(1, 100)
              if random_number >= 70: #more than likely the bot will bet on the river
                post_river_decision = "bet"
              else:
                post_river_decision = "check"

          #bot has a pair on the river after turn and the opponent checks
          elif hand_ranking[0] == 2:
            pair_strength = hand_ranking[5]
            if pair_strength == 1:
              if random.randint(1, 100) > 50:
                post_river_decision = "bet"
              else:
                post_river_decision = "check"
            elif pair_strength == 2:
              if random.randint(1, 100) > 50:
                post_river_decision = "bet" #need to be careful if the opponent is slow playing top pair on the flop
              else:
                post_river_decision = "check"
            else:
              if random.randint(1, 100) > 70: #can potentially be the best hand -- but also can be getting trapped, hopefully opponent folds to this bet
                post_river_decision = "bet"
              else:
                post_river_decision = "check"
          #if the hand ranking is greater then one pair -- can assume the bot flopped an amazing hand and all should play similarly
          else:
            post_river_decision = "bet" #good time to start building a pot with strong hands


          return post_river_decision, hand_ranking[0]

        elif opponent_post_flop_decision == "raise" or opponent_post_turn_decision == "raise":
          #if the bot made it this long with air -- it might be a good idea to start semi bluffing
          if hand_ranking[0] == 1 and flush_draw == False and straight_draw == False:
              random_number = random.randint(1, 100)
              if random_number >= 3: #more than likely the bot will bet on the river
                post_river_decision = "check"
              else:
                post_river_decision = "bet"

          #bot has a pair on the river after turn and the opponent checks
          elif hand_ranking[0] == 2:
            pair_strength = hand_ranking[5]
            if pair_strength == 1:
              if random.randint(1, 100) > 60:
                post_river_decision = "bet"
              else:
                post_river_decision = "check"
            elif pair_strength == 2:
              if random.randint(1, 100) > 75:
                post_river_decision = "bet" #need to be careful if the opponent is slow playing top pair on the flop
              else:
                post_river_decision = "check"
            else:
              if random.randint(1, 100) > 90: #can potentially be the best hand -- but also can be getting trapped, hopefully opponent folds to this bet
                post_river_decision = "bet"
              else:
                post_river_decision = "check"
          #if the hand ranking is greater then one pair -- can assume the bot flopped an amazing hand and all should play similarly
          else:
            random_number = random.randint(1, 100)
            if random_number >= 40:
              post_river_decision = "check"
            else:
              post_river_decision = "bet"

          return post_river_decision, hand_ranking[0]


  def get_post_river_bet_amount(self, pot, hand_ranking): #this function needs to know the potsize to determine sizings
    """
    Bot A post river bet amount
    """

    #at this point the terrible hand should start acting as a bluff
    if hand_ranking == 1:
      random_number = random.randint(1, 100)
      if random_number >= 75:
        post_river_bet_amount = int(round(1/3 * pot)) #large bets to get folds
      elif random_number >= 50:
        post_river_bet_amount = int(round(1/2 * pot))
      elif random_number >= 25:
        post_river_bet_amount = int(round(1 * pot))
      else:
        post_river_bet_amount = int(round(5/4 * pot)) #massive overbet with air to get folds
    #these can be more value bets
    elif hand_ranking == 2:
      random_number = random.randint(1, 100)
      if random_number >= 75:
        post_river_bet_amount = int(round(1/3 * pot))
      elif random_number > 50:
        post_river_bet_amount = int(round(1/2 * pot))
      else:
        post_river_bet_amount = int(round(3/4 * pot))
    else:
      if random.randint(1, 100) > 50:
        post_river_bet_amount = int(round(1/2 * pot))
      else:
        post_river_bet_amount = int(round(5/4 * pot))

    return post_river_bet_amount

  #for the last post river decision its important to mix in some re-raises with weaker hands to have a chance at winning
  def get_post_river_decision_2(self, board):
    """
    This function is for if the other player re-raises a bet, or if the other player bets after bot-a checks on the river
    """
    #this sequence determines what state the players hand is in -- it would be helpful to update the function to recognize board patterns and make more complex decisions
    full_hand = get_seven_card_hand(self, board) #returns a list of the players cards in addition to whats displayed on the board
    hand_ranking = evaluate_hand(full_hand)

    #see if there is more to the bots hand than whats visible
    flush_draw = has_flush_draw(full_hand) #function will return True if the bot needs one card to make a flush
    straight_draw = has_straight_draw(full_hand) #function will return True if the bot needs one card to make a straight

    if hand_ranking[0] == 1 and flush_draw == False and straight_draw == False: #opponent is showing a lot of strength -- fold air
        post_river_decision = "fold"
    #how the bot will act when it turns a pair
    elif hand_ranking[0] == 2: #facing a re-raise or bet here with top pair should skeptically call
      pair_strength = hand_ranking[5]

      if pair_strength == 1:
        random_number = random.randint(1, 100)
        if random_number > 20:
          post_river_decision = "call"
        else:
          post_river_decision = "re-raise"
      elif pair_strength == 2:
        random_number = random.randint(1, 100)
        if random_number >= 65: #
          post_river_decision = "call"
        elif random_number >= 35:
          post_river_decision = "fold"
        else:
          post_river_decision = "re-raise"

    #low pair decision making
      else:
        if random.randint(1, 100) > 10: #low pair almost certianly losing
          post_turn_decision = "fold"
        else:
          post_turn_decision = "call"
    #if the hand ranking is greater then one pair -- logical to defend the hand in most cases with occasional re-raises
    else:
      if random.randint(1, 100) > 25:
        post_turn_decision = "call"
      else:
        post_turn_decision = "re-raise"

      return post_turn_decision, hand_ranking[0]


In [4]:
!git config --global user.name "jmarchese1"

In [5]:
!git config --global user.email "jasonmarchese2002@outlook.com"

In [6]:
%cd /content/poker-ai-headsup

/content/poker-ai-headsup


In [7]:
!pwd

/content/poker-ai-headsup


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [22]:
!git add bots/bot_A.ipynb

In [23]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	engine/__pycache__/

nothing added to commit but untracked files present (use "git add" to track)


In [24]:
!git commit -m "updating bot a file"

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	engine/__pycache__/

nothing added to commit but untracked files present (use "git add" to track)


In [25]:
!git push origin main

Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 6.87 KiB | 6.88 MiB/s, done.
Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
remote: error: GH013: Repository rule violations found for refs/heads/main.
remote: 
remote: - GITHUB PUSH PROTECTION
remote:   —————————————————————————————————————————
remote:     Resolve the following violations before pushing again
remote: 
remote:     - Push cannot contain secrets
remote: 
remote:     
remote:      (?) Learn how to resolve a blocked push
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push
remote:     
remote:     
remote:       —— GitHub Personal Access Token ——————————————————————
remote:        locations:
remote:          - commit: 68287e4415811a7871c0e76852d12ab7c510d7c0
r

In [10]:
!cp "/content/drive/MyDrive/Colab Notebooks/bot_A.ipynb" "/content/poker-ai-headsup/bots/bot_A.ipynb"

In [6]:
cards = create_deck(
)

In [7]:
bot_a = PokerBot("Bot A", 1000, 1, 2)
bot_b = PokerBot("Bot B", 1000, 1, 2)


In [9]:
deal_cards(cards, bot_a, bot_b)

In [10]:
board = []

In [13]:
flop(cards = cards, board = board)

[('5', 'hearts'), ('ace', 'hearts'), ('10', 'diamonds')]


In [14]:
board

[('5', 'hearts'), ('ace', 'hearts'), ('10', 'diamonds')]

In [19]:
full_hand = get_seven_card_hand(bot_a, board)

In [21]:
evaluate_hand(full_hand)[0]

1

In [22]:
bot_a.hand

[('8', 'spades'), ('3', 'clubs')]

In [24]:
has_flush_draw(bot_a.hand, board)

False